In [69]:
import pandas as pd
import numpy as np
import ast
from sklearn.preprocessing import MultiLabelBinarizer

In [70]:
df = pd.read_excel("cars24_v1.xlsx")

In [71]:
df.sample(5)

,appointmentId,listingPrice,carName,make,model,variant,year,transmissionType,bodyType,fuelType,ownership,registrationDate,cityRto,color,odometer,emiDetails,modelGroup,fitnessAge,features
2664,11411786751,534245,Skoda Rapid,Skoda,Rapid,AMBITION 1.6 MPI MT,2019,"{'value': 'Manual', 'display': 'Manual'}",Sedan,Petrol,1,1567296000000,GJ01,Silver,"{'value': 79013, 'display': '79.01k km'}","{'cashDownPayment': 0, 'emiStartingValue': 104...",Rapid,15,"['InfotainmentSystem', 'RearAc', 'AlloyWheels'..."
7272,DLR1000466,435000,Maruti Swift,Maruti,Swift,LXI,2018,"{'value': 'Manual', 'display': 'Manual'}",Hatchback,Petrol,1,1546300800000,JH01,Blue,"{'value': 71000, 'display': '71.00k km'}","{'cashDownPayment': 0, 'emiStartingValue': 850...",Swift,15,[]
1741,23321240791,503000,Tata Tiago,Tata,Tiago,XZA PLUS PETROL,2020,"{'value': 'Automatic', 'display': 'Auto'}",Hatchback,Petrol,1,1593561600000,HR03,White,"{'value': 20036, 'display': '20.04k km'}","{'cashDownPayment': 0, 'emiStartingValue': 983...",Tiago,15,"['InfotainmentSystem', 'AlloyWheels', 'Bluetoo..."
6099,10288548794,872000,Maruti Baleno,Maruti,Baleno,ALPHA PETROL 1.2,2023,"{'value': 'Manual', 'display': 'Manual'}",Hatchback,Petrol,1,1690848000000,KA03,White,"{'value': 9765, 'display': '9.77k km'}","{'cashDownPayment': 0, 'emiStartingValue': 165...",Baleno,15,"['InfotainmentSystem', 'RearAc', 'AlloyWheels'..."
2290,13505048770,704000,Hyundai Verna,Hyundai,Verna,1.6 VTVT SX,2019,"{'value': 'Manual', 'display': 'Manual'}",Sedan,Petrol,1,1577836800000,GJ26,NaN,"{'value': 56230, 'display': '56.23k km'}","{'cashDownPayment': 0, 'emiStartingValue': 137...",Verna,15,"['AlloyWheels', 'Bluetooth', 'ParkingAssist', ..."


**cleaning required**

- `transmission`: contains dictionary, need to extract values
- `registrationDate`: change this to datetime format
- `cityRto`: city info, map this with city
- `odometer`: extract odometer reading
- `emiDetails`: expand this column for more information
- `features`: features of car ( need to process them differently)

##### registrationDate

In [72]:
df['registrationDate']

0       1630454400000
1       1627776000000
2       1706745600000
3       1575158400000
4       1554076800000
            ...      
7386    1527811200000
7387    1682899200000
7388    1575158400000
7389    1627776000000
7390    1646092800000
Name: registrationDate, Length: 7391, dtype: int64

In [73]:
df['registrationDate'] = pd.to_datetime(df['registrationDate'], unit='ms').dt.date

In [74]:
df['registrationDate']

0       2021-09-01
1       2021-08-01
2       2024-02-01
3       2019-12-01
4       2019-04-01
           ...    
7386    2018-06-01
7387    2023-05-01
7388    2019-12-01
7389    2021-08-01
7390    2022-03-01
Name: registrationDate, Length: 7391, dtype: object

##### odometer

In [75]:
df['odometer'].iloc[0]

"{'value': 74426, 'display': '74.43k km'}"

In [76]:
df['odometer'] = df['odometer'].apply(lambda x: ast.literal_eval(x)['value'])

##### emiDetails

In [77]:
df['emiDetails'].iloc[0]

"{'cashDownPayment': 0, 'emiStartingValue': 14956, 'emiEndingValue': 14956, 'roiMinDiscounted': 12.0, 'roiMaxDiscounted': 12.0, 'roiMinOriginal': 12.0, 'roiMaxOriginal': 12.0, 'emiOriginalStartingValue': 14956, 'emiOriginalEndingValue': 14956, 'tenure': 72, 'displayText': 'EMI ₹14,956/m', 'notAvailableText': 'Not available on EMI'}"

In [78]:
df['emiDetails'] = df['emiDetails'].apply(ast.literal_eval)

# Normalize the dictionary into separate columns
emi_df = pd.json_normalize(df['emiDetails'])

# Join back to the original DataFrame (optional)
df = df.drop(columns='emiDetails').join(emi_df)

##### transmission

In [79]:
df['transmissionType'] = df['transmissionType'].apply(lambda x: ast.literal_eval(x)['value'])

##### cityRto

In [80]:
df['cityRto']

0       HR03
1       HR78
2       HR12
3       HR10
4       HR12
        ... 
7386    JH01
7387    JH01
7388    JH05
7389    JH01
7390    JH05
Name: cityRto, Length: 7391, dtype: object

In [81]:
rto_df = pd.read_csv(r'D:\campusx_dsmp2\9. MLOps revisited\cars24_mlops_project\rto_codes.csv')

In [82]:
rto_df['rto_code'] = rto_df['state_name'] + rto_df['city_code'].astype(str).str.zfill(2)

In [83]:
rto_to_city = dict(zip(rto_df['rto_code'], rto_df['city_name']))

In [84]:
df['registeredCity'] = df['cityRto'].map(rto_to_city)

In [85]:
df['registeredCity']

0        PANCHKULA, HR
1         SHAHABAD, HR
2           ROHTAK, HR
3          SONIPAT, HR
4           ROHTAK, HR
             ...      
7386        RANCHI, JH
7387        RANCHI, JH
7388    JAMSHEDPUR, JH
7389        RANCHI, JH
7390    JAMSHEDPUR, JH
Name: registeredCity, Length: 7391, dtype: object

##### features

In [86]:
df['features']

0       ['InfotainmentSystem', 'Bluetooth', 'ParkingAs...
1       ['InfotainmentSystem', 'RearAc', 'Sunroof/Moon...
2                               ['ParkingAssist', 'Tpms']
3                                       ['ParkingAssist']
4                          ['Bluetooth', 'ParkingAssist']
                              ...                        
7386    ['InfotainmentSystem', 'RearAc', 'LeatherSeats...
7387    ['InfotainmentSystem', 'RearAc', 'Sunroof/Moon...
7388    ['InfotainmentSystem', 'RearAc', 'LeatherSeats...
7389    ['InfotainmentSystem', 'RearAc', 'LeatherSeats...
7390    ['InfotainmentSystem', 'RearAc', 'Sunroof/Moon...
Name: features, Length: 7391, dtype: object

In [87]:
import ast
from itertools import chain

# Convert string representations to actual lists
df['features'] = df['features'].apply(ast.literal_eval)

# Now extract all unique features correctly
all_features = set(chain.from_iterable(df['features']))

In [88]:
all_features

{'360DegreeCamera',
 'AlloyWheels',
 'AppleCarplayAndroidAuto',
 'Bluetooth',
 'CruiseControl',
 'GpsNavigation',
 'InfotainmentSystem',
 'LeatherSeats',
 'ParkingAssist',
 'PushButtonStart',
 'RearAc',
 'SpecialRegNo',
 'Sunroof/Moonroof',
 'TopModel',
 'Tpms',
 'VentilatedSeats'}

In [89]:
# Initialize the binarizer
mlb = MultiLabelBinarizer()

# Transform the features list into a binary DataFrame
features_encoded = pd.DataFrame(mlb.fit_transform(df['features']),
                                columns=mlb.classes_,
                                index=df.index)

# Join the encoded features back to the original DataFrame
df = df.join(features_encoded)

In [90]:
df.shape

(7391, 47)

##### export df

In [91]:
df.to_excel("cars24_v2.xlsx", index=False)